In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_hit = pd.read_csv('팀타자_2016.csv')
df_pit = pd.read_csv('팀투수_2016.csv')

df_hit.head()

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,PA,AB,RBI,RUN,...,BB,IB,HP,KK,GD,ERR,LOB,P_HRA_RT,P_AB_CN,P_HIT_CN
0,20160401HHLG0,20160401,LG,HH,0,B,47,42,4,5,...,4,0,0,11,0,0,8,0.333333,12,4
1,20160401HHLG0,20160401,HH,LG,0,T,52,46,4,4,...,3,0,0,10,1,2,12,0.200000,15,3
2,20160401HTNC0,20160401,NC,HT,0,B,36,30,5,5,...,5,0,0,9,1,1,7,0.142857,7,1
3,20160401HTNC0,20160401,HT,NC,0,T,38,34,3,4,...,3,0,1,10,1,0,7,0.100000,10,1
4,20160401KTSK0,20160401,SK,KT,0,B,36,36,4,4,...,0,0,0,7,1,1,5,0.375000,8,3


In [3]:
df_pit.head()

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,CG_CK,WLS,HOLD,INN2,...,KK,GD,WP,BK,ERR,R,ER,P_WHIP_RT,P2_WHIP_RT,CB_WHIP_RT
0,20160401HHLG0,20160401,LG,HH,0,B,0,W,0,36,...,10,1,0,0,0,4,4,0.642857,1.285714,2.400000
1,20160401HHLG0,20160401,HH,LG,0,T,0,L,0,34,...,11,0,1,0,0,5,4,1.500000,1.000000,0.750000
2,20160401HTNC0,20160401,NC,HT,0,B,0,W,0,27,...,10,1,2,0,0,4,4,1.333333,1.038462,2.142857
3,20160401HTNC0,20160401,HT,NC,0,T,0,L,0,24,...,9,1,0,0,0,5,5,0.500000,1.695652,1.875000
4,20160401KTSK0,20160401,SK,KT,0,B,0,L,0,27,...,7,0,0,0,0,8,8,1.000000,2.357143,2.250000


In [4]:
sum(df_hit['G_ID'] == df_pit['G_ID'])

1440

In [5]:
df_mrg = pd.merge(left=df_hit, right=df_pit, how='left', on=['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC'])
df_mrg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1440 entries, 0 to 1439
Data columns (total 56 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   G_ID        1440 non-null   object 
 1   GDAY_DS     1440 non-null   int64  
 2   T_ID        1440 non-null   object 
 3   VS_T_ID     1440 non-null   object 
 4   HEADER_NO   1440 non-null   int64  
 5   TB_SC       1440 non-null   object 
 6   PA_x        1440 non-null   int64  
 7   AB_x        1440 non-null   int64  
 8   RBI         1440 non-null   int64  
 9   RUN         1440 non-null   int64  
 10  HIT_x       1440 non-null   int64  
 11  H2_x        1440 non-null   int64  
 12  H3_x        1440 non-null   int64  
 13  HR_x        1440 non-null   int64  
 14  SB_x        1440 non-null   int64  
 15  CS_x        1440 non-null   int64  
 16  SH_x        1440 non-null   int64  
 17  SF_x        1440 non-null   int64  
 18  BB_x        1440 non-null   int64  
 19  IB_x        1440 non-null  

----
**주요 피처**  
RUN: 경기 내 득점  
R: 경기 내 실점  
WLS: 경기 승무패 (무승부 데이터는 제거해야함)  
____

In [25]:
def cal_sm_1(new_dt):

  PA = new_dt['PA_x'] #타석
  AB = new_dt['AB_x'] #타수
  RBI = new_dt['RBI'] #타점
  RUN = new_dt['RUN'] #득점
  HIT = new_dt['HIT_x'] #총안타
  H2 = new_dt['H2_x'] #2루타
  H3 = new_dt['H3_x'] #3루타
  HR = new_dt['HR_x'] #홈런
  SB = new_dt['SB_x'] #도루
  CS = new_dt['CS_x'] #도루실패
  SH = new_dt['SH_x'] #희생타
  SF = new_dt['SF_x'] #희생플라이
  IB = new_dt['IB_x'] #고의4구
  HP = new_dt['HP_x'] #빈볼
  BB = new_dt['BB_x'] #총사구
  KK = new_dt['KK_x'] #삼진
  GD = new_dt['GD_x'] #병살타
  ERR = new_dt['ERR_x'] #실책
    
  # 순수 1루타 계산
  new_dt['H1_x'] = HIT - H2 - H3 - HR
  H1 = new_dt['H1_x']

  # OBP(출루율)
  new_dt['OBP'] = (HIT+BB+HP)/(AB+BB+SF+HP)
  # SLG(장타율)
  new_dt['SLG'] = (H1+2*H2+3*H3+4*HR)/AB

  # OPS = OBP+SLG
  OBP = new_dt['OBP']
  SLG = new_dt['SLG']
  new_dt['OPS'] = OBP + SLG

  # GPA = (1.8*OBP+SLG)/4 OPS의 단점을 보완
  new_dt['GPA'] = (1.8*new_dt['OBP'] + new_dt['SLG'])/4

  # IsoP = SLG - AVG 순수장타율
  AVG = new_dt['AVG']
  new_dt['IsoP'] = SLG - AVG

  # RC(Run Created)
  new_dt['RC'] = ((HIT+BB-CS+HP-GD) * ((H1+2*H2+3*H3+4*HR)+0.26*(BB-IB+HP)) + (0.52*SH+SF+SB)) / (AB+BB+HP+SH+SF)

  # XR (eXtrapolated Runs, 추정득점(타자의 득점공헌도))
  new_dt['XR'] = (H1*0.5 + H2*0.72 + H3*1.04 + HR*1.44 + (HP+BB-IB)*0.34 + IB*0.25 + SB*0.18 - CS*0.32
                  - (AB-HIT-KK)*0.09 - KK*0.098 - GD*0.37 + SF*0.37 + SH*0.04 )

  # BABIP = (총 안타수-홈런)/(타수-삼진-홈런+희생플라이)
  new_dt['BABIP'] = (HIT-HR)/(AB-KK-HR+SF)

  # OBP = (HIT+BB+HP)/(AB+BB+SF+HP) 출루율
  new_dt['OBP'] = (HIT+BB+HP)/(AB+BB+SF+HP)

  # SLG = (HIT+2*H2+3*H3+4*HR)/AB 장타율
  new_dt['SLG'] = (H1+2*H2+3*H3+4*HR)/AB

  # OPS+ = 100*((OBP/lgOBP+SLG/lgSLG)-1)[/BPF]
  # lgOBP: 리그 평균 출루율 / lgSLG: 리그 평균 장타율
  lgOBP = new_dt['OBP'].mean() # 결측값(NaN) 제외하고 리그 평균 계산
  lgSLG = new_dt['SLG'].mean()
  new_dt['OPS_plus'] = 100*((new_dt['OBP']/lgOBP+new_dt['SLG']/lgSLG)-1)

  # 타수가 극단적으로 적은 타자들로 인해 기록이 부정확할 확률이 높음
  # OPS+ 계산에는 원래 파크팩터를 사용하기도 함. 그러나 시즌 총 기록에서 경기별 패크팩터를 어떻게 대입할지는 모르겠음.

  return new_dt

In [28]:
def cal_sm_2(new_dt):
    
    INN2 = new_dt['INN2']
    BF = new_dt['BF']
    PA = new_dt['PA_y']
    AB = new_dt['AB_y']
    HIT = new_dt['HIT_y']
    H2 = new_dt['H2_y']
    H3 = new_dt['H3_y']
    HR = new_dt['HR_y']
    SH = new_dt['SH_y']
    SF = new_dt['SF_y']
    BB = new_dt['BB_y']
    IB = new_dt['IB_y']
    HP = new_dt['HP_y']
    KK = new_dt['KK_y']
    GD = new_dt['GD_y']
    WP = new_dt['WP']
    BK = new_dt['BK']
    ERR = new_dt['ERR_y']
    R = new_dt['R']
    ER = new_dt['ER']
    
    new_dt['H1_y'] = HIT - H2 - H3 - HR
    new_dt['RA_9'] = (9*new_dt['R']) / (new_dt['INN2']/3)
    new_dt['BABIP'] = (new_dt['HIT_y']-new_dt['HR_y']) / (new_dt['AB_y']-new_dt['KK_y']-new_dt['HR_y']+new_dt['SF_y'])
    new_dt['WHIP'] = (new_dt['HIT_y']+new_dt['BB_y']) /  (new_dt['INN2']/3)
    new_dt['AVG'] = new_dt['HIT_y'] / new_dt['AB_y'] # 피안타율
    new_dt['PTB'] = 0.89*(1.255*(new_dt['HIT_y']-new_dt['HR_y']) + 4*new_dt['HR_y']) + 0.56*(new_dt['BB_y']+new_dt['HP_y']-new_dt['IB_y'])
    new_dt['ERC'] = (((new_dt['HIT_y']+new_dt['BB_y']+new_dt['HP_y'])*new_dt['PTB'])/(new_dt['BF']*(new_dt['INN2']/3)))*9 - 0.56
    
    return new_dt

In [8]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

team_label = encoder.fit_transform(df_mrg['T_ID'])
vs_team_label = encoder.fit_transform(df_mrg['VS_T_ID'])

print(team_label)
print(vs_team_label)

[3 0 5 ... 2 4 9]
[0 3 1 ... 5 9 4]


In [9]:
df_mrg['T_ID'] = team_label
df_mrg['VS_T_ID'] = vs_team_label

df_mrg.tail()

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,PA_x,AB_x,RBI,RUN,...,KK_y,GD_y,WP,BK,ERR_y,R,ER,P_WHIP_RT,P2_WHIP_RT,CB_WHIP_RT
1435,20161008SSSK0,20161008,8,7,0,T,39,36,5,6,...,9,1,0,0,0,7,6,1.5000,2.538462,1.666667
1436,20161009KTNC0,20161009,5,2,0,B,41,39,3,4,...,7,0,2,1,0,7,7,0.9000,2.217391,1.500000
1437,20161009KTNC0,20161009,2,5,0,T,43,38,5,7,...,13,0,0,0,1,4,3,0.5625,1.428571,1.090909
1438,20161009WOLT0,20161009,4,9,0,B,38,36,8,8,...,6,1,1,0,0,5,5,1.5000,1.500000,1.000000
1439,20161009WOLT0,20161009,9,4,0,T,38,33,5,5,...,4,2,1,0,0,8,4,2.2500,1.666667,3.000000


In [10]:
# 쓰이지 않는 피처 제거
df_mrg.drop(['G_ID'], axis=1, inplace=True)
df_mrg.drop(['GDAY_DS'], axis=1, inplace=True)
df_mrg.drop(['HEADER_NO'], axis=1, inplace=True)
df_mrg.drop(['TB_SC'], axis=1, inplace=True)

In [11]:
d_idx = df_mrg[df_mrg['WLS'] == 'D'].index
df_mrg.drop(d_idx, axis=0, inplace=True)
df_mrg.reset_index(drop=True, inplace=True)
df_mrg.shape # 14개 데이터 제거

(1426, 52)

In [12]:
df_mrg['WLS']

0       W
1       L
2       W
3       L
4       L
       ..
1421    L
1422    L
1423    W
1424    W
1425    L
Name: WLS, Length: 1426, dtype: object

In [13]:
df_mrg['WLS'].value_counts()

W    713
L    713
Name: WLS, dtype: int64

In [14]:
for i in range(len(df_mrg)):
    if df_mrg.loc[i, 'WLS'] == 'W':
        df_mrg.loc[i, 'WLS'] = 1
    else:
        df_mrg.loc[i, 'WLS'] = 0

df_mrg['WLS']

0       1
1       0
2       1
3       0
4       0
       ..
1421    0
1422    0
1423    1
1424    1
1425    0
Name: WLS, Length: 1426, dtype: object

In [15]:
df_mrg['WLS'].value_counts()

1    713
0    713
Name: WLS, dtype: int64

In [16]:
#  팀 Set 제작
team = set()
for i in df_mrg['T_ID']:
    team.add(i)
print("2016년 참여 팀 수 :", len(team))

2016년 참여 팀 수 : 10


In [22]:
cl = df_mrg.columns.tolist()
arr = []
df18 = pd.DataFrame(index=range(0,0), columns=cl+['AVG', 'ERA', 'w_rate']) 

for i in team :
    # 선수 별로 새로운 Dataframe 생성
    hitter_df = df_mrg[df_mrg['T_ID'] == i ]
    hitter_df.reset_index(drop=True, inplace=True)
    
    # 경기 데이터가 18개 이상이라면
    count = len(hitter_df)//18
    
    for k in range(count):
        total = hitter_df.loc[k*18:(k*18+17), :].sum()
        total = total / 18
        total['AVG'] = total['HIT_x'] / total['AB_x']
        total['ERA'] = 9*(total['ER'] / (total['INN2']/3))
        total['w_rate'] = total['WLS']

        df18 = df18.append(pd.Series(total, index=df18.columns), ignore_index=True)

        if k == 0 :
            pass
        else :
            arr.append(total['w_rate'])
    arr.append(0)

In [29]:
df18 = cal_sm_1(df18)
df18 = cal_sm_2(df18)
df18.tail()

,T_ID,VS_T_ID,PA_x,AB_x,RBI,RUN,HIT_x,H2_x,H3_x,HR_x,...,IsoP,RC,XR,BABIP,OPS_plus,H1_y,RA_9,WHIP,PTB,ERC
67,9.0,3.388889,38.277778,33.833333,4.166667,4.444444,9.444444,1.388889,0.333333,0.833333,...,0.134647,4.638297,4.785222,0.310000,89.706153,6.777778,5.216561,1.528662,15.949292,0.956284
68,9.0,3.111111,40.111111,35.055556,6.000000,6.444444,10.611111,2.055556,0.277778,0.888889,...,0.150555,5.772659,5.830000,0.335893,106.608316,7.666667,6.444444,1.524691,15.890347,0.893527
69,9.0,3.666667,41.444444,35.166667,6.777778,7.500000,11.222222,2.333333,0.277778,1.166667,...,0.181675,7.040274,6.976667,0.351750,124.940869,8.777778,5.718941,1.698574,18.652081,1.244762
70,9.0,4.777778,39.833333,34.888889,5.055556,5.277778,10.722222,1.611111,0.166667,1.055556,...,0.146497,5.771715,5.870222,0.300797,107.079951,6.777778,4.417178,1.404908,15.472192,0.797481
71,9.0,4.888889,38.944444,33.666667,5.333333,5.555556,9.833333,2.000000,0.166667,0.777778,...,0.138614,5.111401,5.351889,0.358824,98.944094,8.444444,6.339623,1.647799,17.186769,1.148091


In [30]:
df18['label']=arr
df18 = df18[df18['label']!=0]
df18_2 = df18.copy()

df18_2 = df18_2.fillna(0)
df18_2.reset_index(drop=True, inplace=True)

In [31]:
df18[['w_rate', 'label']]

,w_rate,label
0,0.166667,0.333333
1,0.333333,0.722222
2,0.722222,0.388889
3,0.388889,0.666667
4,0.666667,0.444444
...,...,...
66,0.500000,0.500000
67,0.500000,0.555556
68,0.555556,0.722222
69,0.722222,0.555556


In [32]:
df_16_hit = pd.read_csv('팀타자_2016.csv')
df_16_pit = pd.read_csv('팀투수_2016.csv')

df_17_hit = pd.read_csv('팀타자_2017.csv')
df_17_pit = pd.read_csv('팀투수_2017.csv')

df_18_hit = pd.read_csv('팀타자_2018.csv')
df_18_pit = pd.read_csv('팀투수_2018.csv')

df_19_hit = pd.read_csv('팀타자_2019.csv')
df_19_pit = pd.read_csv('팀투수_2019.csv')

df_20_hit = pd.read_csv('팀타자_2020.csv')
df_20_pit = pd.read_csv('팀투수_2020.csv')

In [47]:
def years_preprocessing(df_1, df_2):
    
    df = pd.merge(left=df_1, right=df_2, how='left', on=['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC'])

    # 쓰이지 않는 피처 제거
    df.drop(['G_ID'], axis=1, inplace=True)
    df.drop(['GDAY_DS'], axis=1, inplace=True)
    df.drop(['HEADER_NO'], axis=1, inplace=True)
    df.drop(['TB_SC'], axis=1, inplace=True)

    encoder = LabelEncoder()

    team_label = encoder.fit_transform(df['T_ID'])
    vs_team_label = encoder.fit_transform(df['VS_T_ID'])

    df['T_ID'] = team_label
    df['VS_T_ID'] = vs_team_label

    d_idx = df[df['WLS'] == 'D'].index
    df.drop(d_idx, axis=0, inplace=True)
    df.reset_index(drop=True, inplace=True)

    for i in range(len(df)):
        if df.loc[i, 'WLS'] == 'W':
            df.loc[i, 'WLS'] = 1
        else:
            df.loc[i, 'WLS'] = 0
    
    team = set()
    for i in df['T_ID']:
        team.add(i)

    cl = df.columns.tolist()
    arr = []
    df18 = pd.DataFrame(index=range(0,0), columns=cl+['AVG', 'ERA', 'w_rate']) 

    for i in team :
        # 선수 별로 새로운 Dataframe 생성
        hitter_df = df[df['T_ID'] == i ]
        hitter_df.reset_index(drop=True, inplace=True)

        # 경기 데이터가 18개 이상이라면
        count = len(hitter_df)//18

        for k in range(count):
            total = hitter_df.loc[k*18:(k*18+17), :].sum()
            total = total / 18
            total['AVG'] = total['HIT_x'] / total['AB_x']
            total['ERA'] = 9*(total['ER'] / (total['INN2']/3))
            total['w_rate'] = total['WLS']
            
            df18 = df18.append(pd.Series(total, index=df18.columns), ignore_index=True)

            if k == 0 :
                pass
            else :
                arr.append(total['w_rate'])
        arr.append(0)
        
        
    df18 = cal_sm_2(df18)

    df18['label']=arr
    df18 = df18[df18['label']!=0]
    df18_2 = df18.copy()

    df18_2 = df18_2.fillna(0)
    df18_2.reset_index(drop=True, inplace=True)

    return df18_2

In [48]:
df_16 = years_preprocessing(df_16_hit, df_16_pit)
print('16년 DONE')
df_17 = years_preprocessing(df_17_hit, df_17_pit)
print('17년 DONE')
df_18 = years_preprocessing(df_18_hit, df_18_pit)
print('18년 DONE')
df_19 = years_preprocessing(df_19_hit, df_19_pit)
print('19년 DONE')
df_20 = years_preprocessing(df_20_hit, df_20_pit)
print('20년 DONE')

16년 DONE
17년 DONE
18년 DONE
19년 DONE
20년 DONE


In [49]:
df_all = pd.concat([df_16, df_17, df_18, df_19, df_20])
len(df_all) == len(df_16)+len(df_17)+len(df_18)+len(df_19)+len(df_20)

True

In [50]:
df_all

,T_ID,VS_T_ID,PA_x,AB_x,RBI,RUN,HIT_x,H2_x,H3_x,HR_x,...,AVG,ERA,w_rate,H1_y,RA_9,BABIP,WHIP,PTB,ERC,label
0,0.0,5.333333,39.888889,35.111111,3.500000,3.722222,9.388889,1.666667,0.111111,0.500000,...,0.291401,6.652174,0.166667,6.888889,7.434783,0.334737,1.776398,18.066392,1.182994,0.333333
1,0.0,4.444444,38.944444,33.611111,4.222222,4.500000,8.722222,1.500000,0.055556,0.888889,...,0.310897,6.475375,0.333333,8.333333,7.516060,0.357143,1.920771,18.163681,1.443658,0.722222
2,0.0,5.888889,40.722222,35.944444,6.277778,6.444444,10.777778,1.944444,0.111111,1.166667,...,0.291935,5.334012,0.722222,7.055556,5.718941,0.324544,1.564155,16.508444,0.947984,0.388889
3,0.0,5.444444,39.833333,35.166667,5.388889,5.388889,10.333333,1.444444,0.333333,1.388889,...,0.296063,5.343750,0.388889,7.888889,6.075000,0.333984,1.625000,16.368803,1.003530,0.666667
4,0.0,4.944444,41.000000,35.333333,7.388889,7.611111,10.944444,2.166667,0.444444,1.277778,...,0.281005,5.232990,0.666667,7.388889,5.622680,0.330078,1.521649,14.953586,0.861360,0.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,7.0,3.611111,38.055556,32.666667,4.166667,4.277778,7.833333,1.444444,0.166667,0.944444,...,0.239316,3.905660,0.500000,5.555556,4.132075,0.269912,1.314465,13.681550,0.590463,0.222222
16,8.0,4.333333,37.222222,32.666667,4.666667,4.944444,8.055556,1.833333,0.055556,0.833333,...,0.279675,5.240506,0.333333,6.111111,6.037975,0.307054,1.556962,16.543811,1.005222,0.611111
17,8.0,5.000000,37.444444,33.000000,4.888889,5.055556,8.611111,1.666667,0.055556,1.111111,...,0.243333,3.474684,0.611111,5.500000,4.101266,0.272349,1.303797,13.117803,0.535841,0.666667
18,9.0,3.833333,38.666667,34.000000,5.055556,5.444444,8.777778,2.111111,0.166667,1.000000,...,0.270764,4.101266,0.555556,6.611111,4.670886,0.317125,1.329114,13.590139,0.653355,0.555556


In [51]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [52]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 268 entries, 0 to 19
Data columns (total 62 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   T_ID        268 non-null    float64
 1   VS_T_ID     268 non-null    float64
 2   PA_x        268 non-null    float64
 3   AB_x        268 non-null    float64
 4   RBI         268 non-null    float64
 5   RUN         268 non-null    float64
 6   HIT_x       268 non-null    float64
 7   H2_x        268 non-null    float64
 8   H3_x        268 non-null    float64
 9   HR_x        268 non-null    float64
 10  SB_x        268 non-null    float64
 11  CS_x        268 non-null    float64
 12  SH_x        268 non-null    float64
 13  SF_x        268 non-null    float64
 14  BB_x        268 non-null    float64
 15  IB_x        268 non-null    float64
 16  HP_x        268 non-null    float64
 17  KK_x        268 non-null    float64
 18  GD_x        268 non-null    float64
 19  ERR_x       268 non-null    fl

In [53]:
y_target = df_all['label']
X_data = df_all.drop('label', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.2, random_state=12345)

In [54]:
rf_reg = RandomForestRegressor()
rf_reg.fit(X_train, y_train)
y_pred = rf_reg.predict(X_test)

print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(np.mean(abs(y_pred - y_test)))

0.12967384607444343
0.10267489711934157


In [55]:
xgb_reg = XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_test)

print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(np.mean(abs(y_pred - y_test)))

[16:27:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.13406246335948546
0.10721783008840348


In [56]:
lgb_reg = LGBMRegressor()
lgb_reg.fit(X_train, y_train)
y_pred = lgb_reg.predict(X_test)

print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(np.mean(abs(y_pred - y_test)))

0.12937185523572814
0.10434530965288186


In [57]:
from sklearn.ensemble import IsolationForest

df_all_3 = df_all

clf=IsolationForest(random_state=10, contamination=float(0.04))
clf.fit(df_all_3)

pred = clf.predict(df_all_3)
df_all_3['anomaly']=pred
outliers=df_all_3.loc[df_all_3['anomaly']==-1]
outlier_index=list(outliers.index)
#print(outlier_index)
#Find the number of anomalies and normal points here points classified -1 are anomalous
print(df_all_3['anomaly'].value_counts())
df_all_3 = df_all_3[df_all_3['anomaly']==1]
df_all_3.drop(['anomaly'], axis=1, inplace=True)
df_all_3.shape

 1    257
-1     11
Name: anomaly, dtype: int64


(257, 62)

In [58]:
y_target = df_all_3['label']
X_data = df_all_3.drop('label', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.2, random_state=12345)

In [59]:
rf_reg = RandomForestRegressor()
rf_reg.fit(X_train, y_train)
y_pred = rf_reg.predict(X_test)

print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(np.mean(abs(y_pred - y_test)))

0.14282478690019604
0.11698717948717943


In [60]:
xgb_reg = XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_test)

print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(np.mean(abs(y_pred - y_test)))

[16:28:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.1521011492531793
0.12542145476382005


In [61]:
lgb_reg = LGBMRegressor()
lgb_reg.fit(X_train, y_train)
y_pred = lgb_reg.predict(X_test)

print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(np.mean(abs(y_pred - y_test)))

0.14207866778604122
0.11302808486566877
